# Oficina 4 – Transações em Banco de Dados

Integrantes:
* André Luiz Girão Ferreira
* Sara
* Thaís
* Tiago

**Objetivo**: Experimentar estratégias para utilização de transações e níveis de isolamento em SGBDs relacionais. As tarefas envolvem uma simulação de um sistema de reservas de passagem áreas.








## Informações Iniciais - Simulação de Sistema de Reservas de Passagens

### Tabela e Atributos

Considere a seguinte tabela que registra os assentos reservados em um vôo:

***Assentos(num_voo,disp)***
onde

* ***num_voo***: é um número inteiro de 1 a 200
* ***disp***: é um atributo booleano cujo valor é **true** se o assento estiver vago e **false** caso contrário. **O valor inicial é *true***

### Reserva de Assento

A reserva de um assento é feita em três passos:

* **Passo 1:** O sistema recupera a lista dos assentos disponíveis.
* **Passo 2:** O cliente escolhe o assento. Esse passo deve ser simulado pela escolha aleatória de um dos assentos disponíveis, levando para isso um **tempo de escolha de 1 segundo**.
* **Passo 3:** O sistema registra a reserva do assento escolhido, **atualizando o valor de disp para false**.

Cada assento é reservado individualmente. Duas versões diferentes do processo de reserva devem ser implementadas:
* **Versão A:** A reserva é implementada como uma única transação que inclui os três passos acima.
* **Versão B:** A reserva inclui uma transação para o Passo 1 e outra para o Passo 3. O Passo 2 não faz parte das transações, mas deve ser executado.

### Os Agentes

Agentes de viagens são responsáveis por realizar as reservas de **200 clientes no total**. A atividade de um agente de viagens é simulada por uma ***thread***.

Experimentos devem ser realizados simulando a atuação de **k** agentes de viagem trabalhando simultaneamente, onde
* k = 1,2,4,6,8 e 10.

Cada agente/thread faz uma reserva de cada vez. **As threads devem ser reiniciadas até que todos os 200 clientes tenham seus assentos reservados**.

### Sobre os Experimentos

Dois conjuntos de experimentos devem ser feitos usando dois níveis de isolamento:  
* “read committed”; e
* “serializable”.  

Nos dois casos, o sistema deve ser configurado para **realizar bloqueios a nível de tupla** (linha).


## Tarefas

### Criando tabela no banco de dados

In [3]:
!pip3 install psycopg2
import psycopg2



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import psycopg2

# Conexão com o banco de dados
connection = psycopg2.connect(
    host="localhost",      
    port="5432",           
    database="oficina4",   
    user="root",    
    password="1234"  
)

# Criando um cursor para executar comandos SQL
cursor = connection.cursor()

# Criando a tabela Assentos
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Assentos (
        num_voo INTEGER CHECK (num_voo >= 1 AND num_voo <= 200),
        disp BOOLEAN DEFAULT TRUE
    );
""")

# Salvando a transação
connection.commit()

# Fechando o cursor e a conexão
cursor.close()

### Tarefa 1

Implemente as **versões A e B do processo de reserva**. É importante que as implementações tratem adequadamente conflitos de Concorrência como **deadlocks e rollbacks**

In [ ]:
import threading
import time

In [ ]:
# Função que implementa comportamento do Agente de Viagens
def agente(id, processo_reserva):
    for i in range(3):
        print(f"Agente {id} - está executando {i}")
        processo_reserva()
        time.sleep(1)

In [ ]:
def reservar_acentos(numero_agentes, processo_reserva):
    agentes = []

    # criando threads para agentes
    for id in range(numero_agentes):
        agentes.append(threading.Thread(target=agente, args=(id, processo_reserva)))

    # Iniciando as threads
    for id in range(numero_agentes):
        agentes[id].start()

    # Aguardar o término das threads
    for id in range(numero_agentes):
        agentes[id].join()

    print("Todos os agentes terminaram de trabalhar.")

In [ ]:
def processo_reserva_A():
    pass

def processo_reserva_B():
    pass

In [ ]:
numero_agentes = [1,2,4,6,8,10]  #numero k de agentes

for k in numero_agentes:
    reservar_acentos(k, processo_reserva_A)

for k in numero_agentes:
    reservar_acentos(k, processo_reserva_B)

### Tarefa 2

Apresente gráficos de linha onde, para cada valor de k (número de agentes) no eixo x, temos no eixo y o tempo necessário para que todos os clientes efetuem suas reservas. Um gráfico diferente deve ser apresentado para cada par de versões da reserva e nível de isolamento.



### Tarefa 3

Apresente uma tabela com o número máximo, mínimo e médio de vezes que um cliente teve que tentar reservar um assento até conseguir, ou seja, o número de vezes que uma reserva teve que ser refeita. A tabela considera as variações de k, versão de reserva e nível de isolamento.


### Tarefa 4

Apresente uma análise dos resultados obtidos em cada versão de reserva e tipo de isolamento, explicando as diferenças entre resultados.


### Tarefa 5

Análise de Conflitos de Concorrência. Para cada experimento executado, registre o número de deadlocks e rollbacks detectados pelo sistema gerenciador de banco de dados.

**O que entregar:** Um resumo tabular com o número total de deadlocks e rollbacks para cada combinação de (versão de reserva, nível de isolamento, valor de k). Indicação de como os erros foram tratados no código (ex: tentativas de reexecução, logs etc.).

### Tarefa 6 - Avaliação de Variação na Ordem de Alocação de Assentos

Compare a ordem final dos assentos ocupados (i.e., a sequência de num_voo com disp = false) entre diferentes execuções de um mesmo cenário com concorrência (mesmo k, versão e nível de isolamento), para identificar variações causadas por condições de corrida.

**O que entregar:** Para cada combinação de parâmetros com k > 1, execute o experimento 3 vezes e apresente a variação na ordem dos assentos alocados; Discuta a relação dessa variação com o nível de isolamento e a estrutura transacional usada.



### Tarefa 7 - Demonstração de Anomalias de Concorrência em Diferentes Níveis de Isolamento

O objetivo desta tarefa é observar experimentalmente a ocorrência (ou não) de três tipos clássicos de anomalias em transações concorrentes, sob os níveis de isolamento **READ COMMITTED** e **SERIALIZABLE**. Para isso, implemente três experimentos distintos e controlados, cada um projetado para testar um fenômeno. Cada experimento deve ser executado duas vezes, uma com cada nível de isolamento, e a diferença de comportamento deve ser registrada.



## Experimentos

### Experimento A - Non-repeatable Read

**Descrição:** Uma transação lê um mesmo dado duas vezes, mas entre essas duas leituras, outra transação modifica o dado e realiza commit.

**Cenário:**
* T1 inicia e lê o valor de um assento específico.
* T2 inicia, atualiza o mesmo assento para reservado (disp=false) e comita.
* T1 tenta reler o mesmo assento.

**O que entregar:**
* Código das transações T1 e T2.
* Logs ou saídas indicando se T1 leu valores diferentes na primeira e segunda leitura.
* Análise: isso ocorreu em qual nível de isolamento?




### Experimento B - Phanton Read

**Descrição:** Uma transação executa a mesma consulta duas vezes e obtém conjuntos de resultados diferentes porque outra transação inseriu ou removeu dados que se encaixam no critério da consulta.

**Cenário:**
* T1 inicia e consulta todos os assentos vagos (disp=true).
* T2 inicia, insere um novo assento vago (disp=true) ou atualiza um assento para vago, e comita.
* T1 executa novamente a mesma consulta.


**O que entregar:**
* Código de T1 e T2.
* Saída das duas execuções da consulta de T1.
* Indicação se houve alteração no resultado.
* Discussão da relação com o nível de isolamento usado.


### Experimento C - Dirty Read

**Descrição:** Uma transação lê dados modificados por outra transação que ainda não fez commit (ou que foi revertida). Esse experimento serve para mostrar que o PostgreSQL não permite dirty reads nem mesmo em READ COMMITTED, então o comportamento esperado é que não ocorra a leitura suja.

**Cenário:**
* T1 inicia e atualiza o valor de um assento (disp=false), mas não comita.
* T2 inicia e tenta ler o mesmo assento.
* T1 faz rollback.


**O que entregar:**
* Código de T1 e T2 com sincronização apropriada para simular esse cenário.
* Comprovação de que T2 não teve acesso ao valor não confirmado.
* Discussão: por que isso ocorre mesmo em READ COMMITTED?